<a href="https://colab.research.google.com/github/sungjindo/black_coal_price/blob/main/blackcoal_combine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 기본 라이브러리 및 마운트

In [45]:
# 기본 라이브러리
import pandas as pd
import numpy as np
import os

# 시각화 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib notebook
%matplotlib inline

# 통계 및 기계학습 관련 라이브러리
from scipy import stats    # 통계 관련 라이브러리
from sklearn.model_selection import train_test_split, RandomizedSearchCV    # 데이터 셋 분류 라이브러리
from sklearn.utils import resample    # 데이터 업샘플링&다운샘플링 라이브러리
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTEENN    # 복합샘플링 라이브러리
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler    # 데이터 스케일링 라이브러리
from statsmodels.stats.outliers_influence import variance_inflation_factor    # 변수 간 다중공선성 확인(VIF) 라이브러리
from sklearn.metrics import (accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, auc, plot_confusion_matrix, plot_roc_curve)    # 모델 정확도 검증 라이브러리
from sklearn.model_selection import cross_val_score    # validation  검증 라이브러리
from sklearn.linear_model import LogisticRegression    # 로지스틱 회귀 라이브러리
from sklearn.linear_model import Lasso    # 라쏘 라이브러리
from sklearn.linear_model import Ridge    # 릿지 라이브러리
import statsmodels.api as sm    # 통계분석 관련 라이브러리
from xgboost import XGBClassifier    # XGBoost 라이브러리
from sklearn.ensemble import RandomForestClassifier    # 랜덤포레스트 라이브러리
import warnings    # 경고문구 지우기


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!pip install xlrd

# 엑셀 read

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#DATA

In [ ]:
# df= df.rename(columns={'Unnamed: 0':'date'})

In [ ]:
# # 전체 컬럼명 확인

# #pd.set_option('display.max_seq_items', 10)
# col_names = df.columns.tolist()
# print(df.columns)

Index(['date', 'KPPI', 'ACPI', 'KCPI', 'APPI', 'APPI2'], dtype='object')


In [ ]:
# df1 = df.set_index("date")
# df1

,KPPI,ACPI,KCPI,APPI,APPI2
date,,,,,
2000-01-01,3.1,168.800,1.9,NaN,NaN
2000-02-01,3.0,169.800,1.8,NaN,NaN
2000-03-01,3.2,171.200,2.1,NaN,NaN
2000-04-01,2.4,171.300,1.4,NaN,NaN
2000-05-01,1.7,171.500,1.1,NaN,NaN
...,...,...,...,...,...
2020-08-01,-0.5,259.918,0.8,0.006,123.436202
2020-09-01,-0.4,260.280,0.9,0.003,123.806511
2020-10-01,-0.4,260.388,0.1,0.004,124.301737


In [ ]:
# df_up = df1.resample('W-SUN').ffill()
# df_up

# # 보통 ffill을 사용하거나 보간법을 시행한다.
# # 기간 참고 https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects
# # 개념 참고 https://datascienceschool.net/01%20python/04.08%20%EC%8B%9C%EA%B3%84%EC%97%B4%20%EC%9E%90%EB%A3%8C%20%EB%8B%A4%EB%A3%A8%EA%B8%B0.html


,KPPI,ACPI,KCPI,APPI,APPI2
date,,,,,
2000-01-02,3.1,168.800,1.9,NaN,NaN
2000-01-09,3.1,168.800,1.9,NaN,NaN
2000-01-16,3.1,168.800,1.9,NaN,NaN
2000-01-23,3.1,168.800,1.9,NaN,NaN
2000-01-30,3.1,168.800,1.9,NaN,NaN
...,...,...,...,...,...
2020-11-08,-0.5,260.229,0.6,0.003,124.674642
2020-11-15,-0.5,260.229,0.6,0.003,124.674642
2020-11-22,-0.5,260.229,0.6,0.003,124.674642


In [ ]:
# df_up.to_csv('/content/gdrive/My Drive/Black_coal/data/blackcoalt_m_wtrue1.csv', index = True, encoding = 'utf-8-sig')

In [ ]:
# df2 = pd.read_excel("/content/gdrive/My Drive/Black_coal/data/black_coal.xlsx", sheet_name = [2] )

# df_1 / df_2

In [5]:
df_1 = pd.read_csv("/content/gdrive/My Drive/Black_coal/data/m_w.csv", encoding = 'utf-8-sig')
df_1

,date,KPPI,ACPI,KCPI,APPI,APPI2
0,2008-02-03,5.1,211.693,3.6,0.010,101.000000
1,2008-02-10,5.1,211.693,3.6,0.010,101.000000
2,2008-02-17,5.1,211.693,3.6,0.010,101.000000
3,2008-02-24,5.1,211.693,3.6,0.010,101.000000
4,2008-03-02,6.0,213.528,3.9,0.003,101.303000
...,...,...,...,...,...,...
666,2020-11-08,-0.5,260.229,0.6,0.003,124.674642
667,2020-11-15,-0.5,260.229,0.6,0.003,124.674642
668,2020-11-22,-0.5,260.229,0.6,0.003,124.674642
669,2020-11-29,-0.5,260.229,0.6,0.003,124.674642


In [6]:
df_2 = pd.read_csv("/content/gdrive/My Drive/Black_coal/data/dataset/bdi.csv", encoding = 'utf-8-sig')
df_2

,Date,Price
0,2000-01-03,1318
1,2000-01-04,1320
2,2000-01-05,1329
3,2000-01-06,1351
4,2000-01-07,1368
...,...,...
8172,2022-05-19,3289
8173,2022-05-20,3344
8174,2022-05-21,3344
8175,2022-05-22,3344


In [7]:
df_2= df_2.rename(columns={'Price':'bdi'})

df_2['date']= pd.to_datetime(df_2['Date'])
df_2

,Date,bdi,date
0,2000-01-03,1318,2000-01-03
1,2000-01-04,1320,2000-01-04
2,2000-01-05,1329,2000-01-05
3,2000-01-06,1351,2000-01-06
4,2000-01-07,1368,2000-01-07
...,...,...,...
8172,2022-05-19,3289,2022-05-19
8173,2022-05-20,3344,2022-05-20
8174,2022-05-21,3344,2022-05-21
8175,2022-05-22,3344,2022-05-22


In [8]:
df_2 = df_2.drop(columns=['Date'], axis=1)
df_2

,bdi,date
0,1318,2000-01-03
1,1320,2000-01-04
2,1329,2000-01-05
3,1351,2000-01-06
4,1368,2000-01-07
...,...,...
8172,3289,2022-05-19
8173,3344,2022-05-20
8174,3344,2022-05-21
8175,3344,2022-05-22


In [9]:
df_2 = df_2.set_index("date")
df_2

,bdi
date,
2000-01-03,1318
2000-01-04,1320
2000-01-05,1329
2000-01-06,1351
2000-01-07,1368
...,...
2022-05-19,3289
2022-05-20,3344
2022-05-21,3344


# df_price


In [14]:
df_price = pd.read_csv("/content/gdrive/My Drive/Black_coal/data/가격데이터.csv", encoding = 'utf-8-sig')
df_price

,Unnamed: 0,date,Coal_price,iron,Brent_fut,copper_fut,WTI_fut,WTI,Dubai,LNG_fut
0,0,2022-05-22,408.00,133.99,111.75,9453.5,110.98,112.44,107.17,136.67
1,1,2022-05-15,417.25,133.66,112.55,9422.0,113.23,112.04,107.21,147.96
2,2,2022-05-08,392.85,132.33,111.55,9159.0,110.49,110.30,107.23,148.45
3,3,2022-05-01,379.50,136.91,112.39,9414.5,109.77,110.95,107.57,137.50
4,4,2022-04-24,326.60,150.77,109.34,9769.5,104.69,104.78,102.79,163.68
...,...,...,...,...,...,...,...,...,...,...
698,698,2009-01-04,75.07,NaN,44.42,NaN,40.83,40.92,NaN,56.42
699,699,2008-12-28,74.01,NaN,46.91,NaN,46.34,46.16,NaN,54.74
700,700,2008-12-21,73.09,NaN,38.37,NaN,37.71,37.59,NaN,59.80
701,701,2008-12-14,73.29,NaN,44.00,NaN,33.87,35.89,NaN,59.11


In [15]:
df_price = df_price.drop(columns=['Unnamed: 0'], axis=1)
df_price

,date,Coal_price,iron,Brent_fut,copper_fut,WTI_fut,WTI,Dubai,LNG_fut
0,2022-05-22,408.00,133.99,111.75,9453.5,110.98,112.44,107.17,136.67
1,2022-05-15,417.25,133.66,112.55,9422.0,113.23,112.04,107.21,147.96
2,2022-05-08,392.85,132.33,111.55,9159.0,110.49,110.30,107.23,148.45
3,2022-05-01,379.50,136.91,112.39,9414.5,109.77,110.95,107.57,137.50
4,2022-04-24,326.60,150.77,109.34,9769.5,104.69,104.78,102.79,163.68
...,...,...,...,...,...,...,...,...,...
698,2009-01-04,75.07,NaN,44.42,NaN,40.83,40.92,NaN,56.42
699,2008-12-28,74.01,NaN,46.91,NaN,46.34,46.16,NaN,54.74
700,2008-12-21,73.09,NaN,38.37,NaN,37.71,37.59,NaN,59.80
701,2008-12-14,73.29,NaN,44.00,NaN,33.87,35.89,NaN,59.11


In [16]:
df_price = df_price.set_index('date')
df_price

,Coal_price,iron,Brent_fut,copper_fut,WTI_fut,WTI,Dubai,LNG_fut
date,,,,,,,,
2022-05-22,408.00,133.99,111.75,9453.5,110.98,112.44,107.17,136.67
2022-05-15,417.25,133.66,112.55,9422.0,113.23,112.04,107.21,147.96
2022-05-08,392.85,132.33,111.55,9159.0,110.49,110.30,107.23,148.45
2022-05-01,379.50,136.91,112.39,9414.5,109.77,110.95,107.57,137.50
2022-04-24,326.60,150.77,109.34,9769.5,104.69,104.78,102.79,163.68
...,...,...,...,...,...,...,...,...
2009-01-04,75.07,NaN,44.42,NaN,40.83,40.92,NaN,56.42
2008-12-28,74.01,NaN,46.91,NaN,46.34,46.16,NaN,54.74
2008-12-21,73.09,NaN,38.37,NaN,37.71,37.59,NaN,59.80


## df_combine

In [17]:
df_left = pd.merge(df_price, df_1, left_on='date', right_on='date', how='left')
print(df_left)

           date  Coal_price    iron  Brent_fut  copper_fut  WTI_fut     WTI  \
0    2022-05-22      408.00  133.99     111.75      9453.5   110.98  112.44   
1    2022-05-15      417.25  133.66     112.55      9422.0   113.23  112.04   
2    2022-05-08      392.85  132.33     111.55      9159.0   110.49  110.30   
3    2022-05-01      379.50  136.91     112.39      9414.5   109.77  110.95   
4    2022-04-24      326.60  150.77     109.34      9769.5   104.69  104.78   
..          ...         ...     ...        ...         ...      ...     ...   
698  2009-01-04       75.07     NaN      44.42         NaN    40.83   40.92   
699  2008-12-28       74.01     NaN      46.91         NaN    46.34   46.16   
700  2008-12-21       73.09     NaN      38.37         NaN    37.71   37.59   
701  2008-12-14       73.29     NaN      44.00         NaN    33.87   35.89   
702  2008-12-07       74.04     NaN      46.41         NaN    46.28   46.37   

      Dubai  LNG_fut  KPPI     ACPI  KCPI   APPI   

In [18]:
df_left.isnull().sum()

date            0
Coal_price      0
iron           98
Brent_fut       0
copper_fut     84
WTI_fut         0
WTI             0
Dubai         281
LNG_fut         1
KPPI           76
ACPI           76
KCPI           76
APPI           76
APPI2          76
dtype: int64

In [20]:
df_left['date']= pd.to_datetime(df_left['date'])

df_left = df_left.set_index('date')
df_left

,Coal_price,iron,Brent_fut,copper_fut,WTI_fut,WTI,Dubai,LNG_fut,KPPI,ACPI,KCPI,APPI,APPI2
date,,,,,,,,,,,,,
2022-05-22,408.00,133.99,111.75,9453.5,110.98,112.44,107.17,136.67,NaN,NaN,NaN,NaN,NaN
2022-05-15,417.25,133.66,112.55,9422.0,113.23,112.04,107.21,147.96,NaN,NaN,NaN,NaN,NaN
2022-05-08,392.85,132.33,111.55,9159.0,110.49,110.30,107.23,148.45,NaN,NaN,NaN,NaN,NaN
2022-05-01,379.50,136.91,112.39,9414.5,109.77,110.95,107.57,137.50,NaN,NaN,NaN,NaN,NaN
2022-04-24,326.60,150.77,109.34,9769.5,104.69,104.78,102.79,163.68,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009-01-04,75.07,NaN,44.42,NaN,40.83,40.92,NaN,56.42,4.6,211.143,3.7,-0.019,98.676685
2008-12-28,74.01,NaN,46.91,NaN,46.34,46.16,NaN,54.74,5.5,210.228,4.1,-0.022,100.587854
2008-12-21,73.09,NaN,38.37,NaN,37.71,37.59,NaN,59.80,5.5,210.228,4.1,-0.022,100.587854


In [21]:
# df_LEFT_JOIN_2 = df_LEFT_JOIN.join(df_2, how = 'left') #디폴트는 이너 
# print(df_LEFT_JOIN_2)

# # Index 기준 합칠땐 join
# # merge는 단순 합치기

df_left2 = pd.merge(df_left, df_2, left_index = True ,right_index = True, how='left')
print(df_left2)

            Coal_price    iron  Brent_fut  copper_fut  WTI_fut     WTI  \
date                                                                     
2022-05-22      408.00  133.99     111.75      9453.5   110.98  112.44   
2022-05-15      417.25  133.66     112.55      9422.0   113.23  112.04   
2022-05-08      392.85  132.33     111.55      9159.0   110.49  110.30   
2022-05-01      379.50  136.91     112.39      9414.5   109.77  110.95   
2022-04-24      326.60  150.77     109.34      9769.5   104.69  104.78   
...                ...     ...        ...         ...      ...     ...   
2009-01-04       75.07     NaN      44.42         NaN    40.83   40.92   
2008-12-28       74.01     NaN      46.91         NaN    46.34   46.16   
2008-12-21       73.09     NaN      38.37         NaN    37.71   37.59   
2008-12-14       73.29     NaN      44.00         NaN    33.87   35.89   
2008-12-07       74.04     NaN      46.41         NaN    46.28   46.37   

             Dubai  LNG_fut  KPPI    

In [22]:
# df_2 = df_2.sort_values('date', ascending= False)
# df_2

In [23]:
df_2.index.dtype

dtype('<M8[ns]')

In [25]:
df_left2.index.dtype

dtype('<M8[ns]')

In [26]:
df_left2

,Coal_price,iron,Brent_fut,copper_fut,WTI_fut,WTI,Dubai,LNG_fut,KPPI,ACPI,KCPI,APPI,APPI2,bdi
date,,,,,,,,,,,,,,
2022-05-22,408.00,133.99,111.75,9453.5,110.98,112.44,107.17,136.67,NaN,NaN,NaN,NaN,NaN,3344
2022-05-15,417.25,133.66,112.55,9422.0,113.23,112.04,107.21,147.96,NaN,NaN,NaN,NaN,NaN,3104
2022-05-08,392.85,132.33,111.55,9159.0,110.49,110.30,107.23,148.45,NaN,NaN,NaN,NaN,NaN,2718
2022-05-01,379.50,136.91,112.39,9414.5,109.77,110.95,107.57,137.50,NaN,NaN,NaN,NaN,NaN,2404
2022-04-24,326.60,150.77,109.34,9769.5,104.69,104.78,102.79,163.68,NaN,NaN,NaN,NaN,NaN,2307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009-01-04,75.07,NaN,44.42,NaN,40.83,40.92,NaN,56.42,4.6,211.143,3.7,-0.019,98.676685,773
2008-12-28,74.01,NaN,46.91,NaN,46.34,46.16,NaN,54.74,5.5,210.228,4.1,-0.022,100.587854,774
2008-12-21,73.09,NaN,38.37,NaN,37.71,37.59,NaN,59.80,5.5,210.228,4.1,-0.022,100.587854,818


In [27]:
df_left2.isnull().sum()

Coal_price      0
iron           98
Brent_fut       0
copper_fut     84
WTI_fut         0
WTI             0
Dubai         281
LNG_fut         1
KPPI           76
ACPI           76
KCPI           76
APPI           76
APPI2          76
bdi             0
dtype: int64

# Electronic

In [28]:
df_ele = pd.read_csv("/content/gdrive/My Drive/Black_coal/data/ele_pro.csv", encoding = 'utf-8-sig')
df_ele

,date,South Korea_Electricity from coal (TWh),South Korea_Electricity from gas (TWh),South KoreaElectricity from hydro (TWh),South Korea_Electricity from other renewables including bioenergy (TWh),South Korea_Electricity from solar (TWh),South Korea_Electricity from oil (TWh),South Korea_Electricity from wind (TWh),South Korea_Electricity from nuclear (TWh),Japan_Electricity from coal (TWh),...,China_Electricity from wind (TWh),China_Electricity from nuclear (TWh),India_Electricity from coal (TWh),India_Electricity from gas (TWh),India_Electricity from hydro (TWh),India_Electricity from other renewables including bioenergy (TWh),India_Electricity from solar (TWh),India_Electricity from oil (TWh),India_Electricity from wind (TWh),India_Electricity from nuclear (TWh)
0,2000-01-01,107.49,28.18,3.97,0.08,0.00,29.26,0.02,103.52,204.84,...,0.59,16.74,390.23,55.96,76.99,1.686,0.01,29.16,1.58,15.77
1,2000-01-02,107.49,28.18,3.97,0.08,0.00,29.26,0.02,103.52,204.84,...,0.59,16.74,390.23,55.96,76.99,1.686,0.01,29.16,1.58,15.77
2,2000-01-03,107.49,28.18,3.97,0.08,0.00,29.26,0.02,103.52,204.84,...,0.59,16.74,390.23,55.96,76.99,1.686,0.01,29.16,1.58,15.77
3,2000-01-04,107.49,28.18,3.97,0.08,0.00,29.26,0.02,103.52,204.84,...,0.59,16.74,390.23,55.96,76.99,1.686,0.01,29.16,1.58,15.77
4,2000-01-05,107.49,28.18,3.97,0.08,0.00,29.26,0.02,103.52,204.84,...,0.59,16.74,390.23,55.96,76.99,1.686,0.01,29.16,1.58,15.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8031,2021-12-27,199.05,167.92,6.16,22.82,23.71,2.27,3.17,150.14,296.65,...,614.18,407.47,1250.46,58.11,171.27,0.000,68.34,4.86,68.03,42.67
8032,2021-12-28,199.05,167.92,6.16,22.82,23.71,2.27,3.17,150.14,296.65,...,614.18,407.47,1250.46,58.11,171.27,0.000,68.34,4.86,68.03,42.67
8033,2021-12-29,199.05,167.92,6.16,22.82,23.71,2.27,3.17,150.14,296.65,...,614.18,407.47,1250.46,58.11,171.27,0.000,68.34,4.86,68.03,42.67
8034,2021-12-30,199.05,167.92,6.16,22.82,23.71,2.27,3.17,150.14,296.65,...,614.18,407.47,1250.46,58.11,171.27,0.000,68.34,4.86,68.03,42.67


In [29]:
df_ele.isnull().sum()

date                                                                       0
South Korea_Electricity from coal (TWh)                                    0
South Korea_Electricity from gas (TWh)                                     0
South KoreaElectricity from hydro (TWh)                                    0
South Korea_Electricity from other renewables including bioenergy (TWh)    0
South Korea_Electricity from solar (TWh)                                   0
South Korea_Electricity from oil (TWh)                                     0
South Korea_Electricity from wind (TWh)                                    0
South Korea_Electricity from nuclear (TWh)                                 0
Japan_Electricity from coal (TWh)                                          0
Japan_Electricity from gas (TWh)                                           0
Japan_Electricity from hydro (TWh)                                         0
Japan_Electricity from other renewables including bioenergy (TWh)          0

In [30]:
df_ele['date']= pd.to_datetime(df_ele['date'])

df_ele = df_ele.set_index('date')
df_ele

,South Korea_Electricity from coal (TWh),South Korea_Electricity from gas (TWh),South KoreaElectricity from hydro (TWh),South Korea_Electricity from other renewables including bioenergy (TWh),South Korea_Electricity from solar (TWh),South Korea_Electricity from oil (TWh),South Korea_Electricity from wind (TWh),South Korea_Electricity from nuclear (TWh),Japan_Electricity from coal (TWh),Japan_Electricity from gas (TWh),...,China_Electricity from wind (TWh),China_Electricity from nuclear (TWh),India_Electricity from coal (TWh),India_Electricity from gas (TWh),India_Electricity from hydro (TWh),India_Electricity from other renewables including bioenergy (TWh),India_Electricity from solar (TWh),India_Electricity from oil (TWh),India_Electricity from wind (TWh),India_Electricity from nuclear (TWh)
date,,,,,,,,,,,,,,,,,,,,,
2000-01-01,107.49,28.18,3.97,0.08,0.00,29.26,0.02,103.52,204.84,222.25,...,0.59,16.74,390.23,55.96,76.99,1.686,0.01,29.16,1.58,15.77
2000-01-02,107.49,28.18,3.97,0.08,0.00,29.26,0.02,103.52,204.84,222.25,...,0.59,16.74,390.23,55.96,76.99,1.686,0.01,29.16,1.58,15.77
2000-01-03,107.49,28.18,3.97,0.08,0.00,29.26,0.02,103.52,204.84,222.25,...,0.59,16.74,390.23,55.96,76.99,1.686,0.01,29.16,1.58,15.77
2000-01-04,107.49,28.18,3.97,0.08,0.00,29.26,0.02,103.52,204.84,222.25,...,0.59,16.74,390.23,55.96,76.99,1.686,0.01,29.16,1.58,15.77
2000-01-05,107.49,28.18,3.97,0.08,0.00,29.26,0.02,103.52,204.84,222.25,...,0.59,16.74,390.23,55.96,76.99,1.686,0.01,29.16,1.58,15.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,199.05,167.92,6.16,22.82,23.71,2.27,3.17,150.14,296.65,310.39,...,614.18,407.47,1250.46,58.11,171.27,0.000,68.34,4.86,68.03,42.67
2021-12-28,199.05,167.92,6.16,22.82,23.71,2.27,3.17,150.14,296.65,310.39,...,614.18,407.47,1250.46,58.11,171.27,0.000,68.34,4.86,68.03,42.67
2021-12-29,199.05,167.92,6.16,22.82,23.71,2.27,3.17,150.14,296.65,310.39,...,614.18,407.47,1250.46,58.11,171.27,0.000,68.34,4.86,68.03,42.67


In [31]:
df_left2

,Coal_price,iron,Brent_fut,copper_fut,WTI_fut,WTI,Dubai,LNG_fut,KPPI,ACPI,KCPI,APPI,APPI2,bdi
date,,,,,,,,,,,,,,
2022-05-22,408.00,133.99,111.75,9453.5,110.98,112.44,107.17,136.67,NaN,NaN,NaN,NaN,NaN,3344
2022-05-15,417.25,133.66,112.55,9422.0,113.23,112.04,107.21,147.96,NaN,NaN,NaN,NaN,NaN,3104
2022-05-08,392.85,132.33,111.55,9159.0,110.49,110.30,107.23,148.45,NaN,NaN,NaN,NaN,NaN,2718
2022-05-01,379.50,136.91,112.39,9414.5,109.77,110.95,107.57,137.50,NaN,NaN,NaN,NaN,NaN,2404
2022-04-24,326.60,150.77,109.34,9769.5,104.69,104.78,102.79,163.68,NaN,NaN,NaN,NaN,NaN,2307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009-01-04,75.07,NaN,44.42,NaN,40.83,40.92,NaN,56.42,4.6,211.143,3.7,-0.019,98.676685,773
2008-12-28,74.01,NaN,46.91,NaN,46.34,46.16,NaN,54.74,5.5,210.228,4.1,-0.022,100.587854,774
2008-12-21,73.09,NaN,38.37,NaN,37.71,37.59,NaN,59.80,5.5,210.228,4.1,-0.022,100.587854,818


In [32]:
# df_LEFT_JOIN_2 = df_LEFT_JOIN.join(df_2, how = 'left') #디폴트는 이너 
# print(df_LEFT_JOIN_2)

# # Index 기준 합칠땐 join
# # merge는 단순 합치기

df_left3 = pd.merge(df_left2, df_ele , left_index = True ,right_index = True, how='left')
print(df_left3)

            Coal_price    iron  Brent_fut  copper_fut  WTI_fut     WTI  \
date                                                                     
2022-05-22      408.00  133.99     111.75      9453.5   110.98  112.44   
2022-05-15      417.25  133.66     112.55      9422.0   113.23  112.04   
2022-05-08      392.85  132.33     111.55      9159.0   110.49  110.30   
2022-05-01      379.50  136.91     112.39      9414.5   109.77  110.95   
2022-04-24      326.60  150.77     109.34      9769.5   104.69  104.78   
...                ...     ...        ...         ...      ...     ...   
2009-01-04       75.07     NaN      44.42         NaN    40.83   40.92   
2008-12-28       74.01     NaN      46.91         NaN    46.34   46.16   
2008-12-21       73.09     NaN      38.37         NaN    37.71   37.59   
2008-12-14       73.29     NaN      44.00         NaN    33.87   35.89   
2008-12-07       74.04     NaN      46.41         NaN    46.28   46.37   

             Dubai  LNG_fut  KPPI    

In [33]:
df_left3

,Coal_price,iron,Brent_fut,copper_fut,WTI_fut,WTI,Dubai,LNG_fut,KPPI,ACPI,...,China_Electricity from wind (TWh),China_Electricity from nuclear (TWh),India_Electricity from coal (TWh),India_Electricity from gas (TWh),India_Electricity from hydro (TWh),India_Electricity from other renewables including bioenergy (TWh),India_Electricity from solar (TWh),India_Electricity from oil (TWh),India_Electricity from wind (TWh),India_Electricity from nuclear (TWh)
date,,,,,,,,,,,,,,,,,,,,,
2022-05-22,408.00,133.99,111.75,9453.5,110.98,112.44,107.17,136.67,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-15,417.25,133.66,112.55,9422.0,113.23,112.04,107.21,147.96,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-08,392.85,132.33,111.55,9159.0,110.49,110.30,107.23,148.45,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-01,379.50,136.91,112.39,9414.5,109.77,110.95,107.57,137.50,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-04-24,326.60,150.77,109.34,9769.5,104.69,104.78,102.79,163.68,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009-01-04,75.07,NaN,44.42,NaN,40.83,40.92,NaN,56.42,4.6,211.143,...,27.61,70.05,611.11,104.37,106.34,11.642,0.08,13.08,16.27,16.82
2008-12-28,74.01,NaN,46.91,NaN,46.34,46.16,NaN,54.74,5.5,210.228,...,13.10,68.39,573.12,85.88,115.17,9.302,0.06,15.27,14.38,15.23
2008-12-21,73.09,NaN,38.37,NaN,37.71,37.59,NaN,59.80,5.5,210.228,...,13.10,68.39,573.12,85.88,115.17,9.302,0.06,15.27,14.38,15.23


In [35]:
df_left3.isnull().sum()

Coal_price                                                                   0
iron                                                                        98
Brent_fut                                                                    0
copper_fut                                                                  84
WTI_fut                                                                      0
WTI                                                                          0
Dubai                                                                      281
LNG_fut                                                                      1
KPPI                                                                        76
ACPI                                                                        76
KCPI                                                                        76
APPI                                                                        76
APPI2                                               

In [39]:
df_f = df_left3.query('"2011-01-01"<= date <= "2019-12-31"')

In [40]:
df_f

,Coal_price,iron,Brent_fut,copper_fut,WTI_fut,WTI,Dubai,LNG_fut,KPPI,ACPI,...,China_Electricity from wind (TWh),China_Electricity from nuclear (TWh),India_Electricity from coal (TWh),India_Electricity from gas (TWh),India_Electricity from hydro (TWh),India_Electricity from other renewables including bioenergy (TWh),India_Electricity from solar (TWh),India_Electricity from oil (TWh),India_Electricity from wind (TWh),India_Electricity from nuclear (TWh)
date,,,,,,,,,,,,,,,,,,,,,
2019-12-29,67.50,93.41,68.60,6129.5,63.05,63.10,67.83,33.07,0.7,256.974,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
2019-12-22,66.55,91.55,68.16,6214.0,61.72,61.74,64.82,33.13,0.7,256.974,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
2019-12-15,66.35,91.74,66.14,6175.0,60.44,60.44,64.63,37.12,0.7,256.974,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
2019-12-08,66.30,92.27,65.22,6130.0,60.07,59.78,64.12,35.17,0.7,256.974,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
2019-12-01,66.40,88.56,64.39,5990.0,59.20,59.11,63.43,39.05,0.7,256.974,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-01-30,127.00,183.50,99.83,10067.0,89.03,89.13,NaN,54.84,6.8,220.223,...,74.1,87.2,701.25,117.56,131.70,17.137,0.83,9.35,23.95,32.22
2011-01-23,130.35,179.50,99.42,9548.5,89.34,89.49,NaN,53.73,6.8,220.223,...,74.1,87.2,701.25,117.56,131.70,17.137,0.83,9.35,23.95,32.22
2011-01-16,131.90,178.00,97.60,9462.5,89.11,89.13,NaN,55.12,6.8,220.223,...,74.1,87.2,701.25,117.56,131.70,17.137,0.83,9.35,23.95,32.22


In [41]:
df_f.isnull().sum()

Coal_price                                                                   0
iron                                                                         0
Brent_fut                                                                    0
copper_fut                                                                   0
WTI_fut                                                                      0
WTI                                                                          0
Dubai                                                                      173
LNG_fut                                                                      0
KPPI                                                                         0
ACPI                                                                         0
KCPI                                                                         0
APPI                                                                         0
APPI2                                               

In [42]:
df_f = df_f.drop('Dubai', axis = 1)
df_f

,Coal_price,iron,Brent_fut,copper_fut,WTI_fut,WTI,LNG_fut,KPPI,ACPI,KCPI,...,China_Electricity from wind (TWh),China_Electricity from nuclear (TWh),India_Electricity from coal (TWh),India_Electricity from gas (TWh),India_Electricity from hydro (TWh),India_Electricity from other renewables including bioenergy (TWh),India_Electricity from solar (TWh),India_Electricity from oil (TWh),India_Electricity from wind (TWh),India_Electricity from nuclear (TWh)
date,,,,,,,,,,,,,,,,,,,,,
2019-12-29,67.50,93.41,68.60,6129.5,63.05,63.10,33.07,0.7,256.974,0.7,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
2019-12-22,66.55,91.55,68.16,6214.0,61.72,61.74,33.13,0.7,256.974,0.7,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
2019-12-15,66.35,91.74,66.14,6175.0,60.44,60.44,37.12,0.7,256.974,0.7,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
2019-12-08,66.30,92.27,65.22,6130.0,60.07,59.78,35.17,0.7,256.974,0.7,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
2019-12-01,66.40,88.56,64.39,5990.0,59.20,59.11,39.05,0.7,256.974,0.7,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-01-30,127.00,183.50,99.83,10067.0,89.03,89.13,54.84,6.8,220.223,3.4,...,74.1,87.2,701.25,117.56,131.70,17.137,0.83,9.35,23.95,32.22
2011-01-23,130.35,179.50,99.42,9548.5,89.34,89.49,53.73,6.8,220.223,3.4,...,74.1,87.2,701.25,117.56,131.70,17.137,0.83,9.35,23.95,32.22
2011-01-16,131.90,178.00,97.60,9462.5,89.11,89.13,55.12,6.8,220.223,3.4,...,74.1,87.2,701.25,117.56,131.70,17.137,0.83,9.35,23.95,32.22


In [44]:
df_f.isnull().sum()

Coal_price                                                                 0
iron                                                                       0
Brent_fut                                                                  0
copper_fut                                                                 0
WTI_fut                                                                    0
WTI                                                                        0
LNG_fut                                                                    0
KPPI                                                                       0
ACPI                                                                       0
KCPI                                                                       0
APPI                                                                       0
APPI2                                                                      0
bdi                                                                        0